## 네이버 로컬 API 활용

In [43]:
import requests, json
import urllib
from urllib.request import Request, urlopen

- API Key

In [44]:
with open('keys/네이버apikey.txt') as file:
    naver_key = file.read()

- URL 만들기

In [71]:
start = '서울특별시 영등포구 영중로8길 6 401(영등포동3가)'
goal = '경기도 고양시 덕양구 소원로 102'
waypoint = '경기도 수원시 장안구 서부로 2149'

In [72]:
# 주소에 geocoding 적용하는 함수를 작성.
def get_location(loc) :
    client_id = 'ogam984n5u'
    client_secret = naver_key
    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(loc)
    
    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return한다
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        print(response_body)
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 : 
        	# 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lon = response_body['addresses'][0]['x']
            return (lon, lat)
        else :
            print('location not exist')
        
    else :
        print('ERROR')
        
#  함수 적용
start = get_location(start)
goal = get_location(goal)
waypoint = get_location(waypoint)

{'status': 'OK', 'meta': {'totalCount': 1, 'page': 1, 'count': 1}, 'addresses': [{'roadAddress': '서울특별시 영등포구 영중로8길 6 성남빌딩', 'jibunAddress': '서울특별시 영등포구 영등포동3가 8-1 성남빌딩', 'englishAddress': '6, Yeongjung-ro 8-gil, Yeongdeungpo-gu, Seoul, Republic of Korea', 'addressElements': [{'types': ['SIDO'], 'longName': '서울특별시', 'shortName': '서울특별시', 'code': ''}, {'types': ['SIGUGUN'], 'longName': '영등포구', 'shortName': '영등포구', 'code': ''}, {'types': ['DONGMYUN'], 'longName': '영등포동3가', 'shortName': '영등포동3가', 'code': ''}, {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['ROAD_NAME'], 'longName': '영중로8길', 'shortName': '영중로8길', 'code': ''}, {'types': ['BUILDING_NUMBER'], 'longName': '6', 'shortName': '6', 'code': ''}, {'types': ['BUILDING_NAME'], 'longName': '성남빌딩', 'shortName': '성남빌딩', 'code': ''}, {'types': ['LAND_NUMBER'], 'longName': '8-1', 'shortName': '8-1', 'code': ''}, {'types': ['POSTAL_CODE'], 'longName': '07302', 'shortName': '07302', 'code': ''}], 'x': '126.9061910',

In [73]:
start, goal, waypoint

(('126.9061910', '37.5183657'),
 ('126.8341891', '37.6121307'),
 ('126.9721787', '37.3001965'))

In [84]:
start, goal = (127.1058342, 37.3597078), (129.0759853, 35.1794697)

In [85]:
option = 'traoptimal'
client_id = 'ogam984n5u'
client_secret = naver_key
url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start}&goal={goal}&option={option}"
        

header = {'X-NCP-APIGW-API-KEY-ID': client_id, 'X-NCP-APIGW-API-KEY': naver_key}

result = requests.get(url, headers=header).json()

In [86]:
result

{'code': 2, 'message': '출발지 또는 도착지가 도로 주변이 아닙니다. 위치를 변경해 주세요.'}

In [90]:
import requests

def get_coordinates(location):
    base_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': 'ogam984n5u',  # 여기에 본인의 API 키 ID 입력
        'X-NCP-APIGW-API-KEY': naver_key         # 여기에 본인의 API 키 입력
    }

    params = {
        'query': location
    }

    response = requests.get(base_url, headers=headers, params=params)
    json_data = response.json()

    if 'addresses' in json_data and json_data['addresses']:
        x = json_data['addresses'][0]['x']
        y = json_data['addresses'][0]['y']
        print(f"{location}의 위도는 {y}, 경도는 {x} 입니다.")
        return x, y
    else:
        print(f"{location}의 좌표를 찾을 수 없습니다.")
        return None, None

def get_directions(start, end):
    start_x, start_y = get_coordinates(start)
    end_x, end_y = get_coordinates(end)

    if start_x is None or end_x is None:
        return

    base_url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': 'ogam984n5u',  # 여기에 본인의 API 키 ID 입력
        'X-NCP-APIGW-API-KEY': naver_key         # 여기에 본인의 API 키 입력
    }

    params = {
        'start': f"{start_x},{start_y}",
        'goal': f"{end_x},{end_y}"
    }

    response = requests.get(base_url, headers=headers, params=params)
    json_data = response.json()

    if 'route' in json_data:
        distance = json_data['route']['trafast'][0]['summary']['distance']
        print(f"출발지에서 도착지까지의 거리는 {distance}m 입니다.")
        print("경로 정보:")
        for path in json_data['route']['trafast'][0]['path']:
            print(path)
    else:
        print("경로를 찾을 수 없습니다.")

# 출발지와 도착지 설정 (주소)
start_address = '서울특별시 강남구 삼성동'  # 예시: '서울특별시 강남구 삼성동'
end_address = '경기도 고양시 덕양구 마상로 164'    # 예시: '서울특별시 중구 을지로'

get_directions(start_address, end_address)


서울특별시 강남구 삼성동의 위도는 37.514442, 경도는 127.062532 입니다.
경기도 고양시 덕양구 마상로 164의 위도는 37.6578214, 경도는 126.8403659 입니다.


KeyError: 'trafast'

In [92]:
import requests

def get_coordinates(location):
    base_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': 'ogam984n5u',  # 여기에 본인의 API 키 ID 입력
        'X-NCP-APIGW-API-KEY': naver_key         # 여기에 본인의 API 키 입력
    }

    params = {
        'query': location
    }

    response = requests.get(base_url, headers=headers, params=params)
    json_data = response.json()

    if 'addresses' in json_data and json_data['addresses']:
        x = json_data['addresses'][0]['x']
        y = json_data['addresses'][0]['y']
        print(f"{location}의 위도는 {y}, 경도는 {x} 입니다.")
        return x, y
    else:
        print(f"{location}의 좌표를 찾을 수 없습니다.")
        return None, None

def get_directions(start, end):
    start_x, start_y = get_coordinates(start)
    end_x, end_y = get_coordinates(end)

    if start_x is None or end_x is None:
        return

    base_url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': 'ogam984n5u',  # 여기에 본인의 API 키 ID 입력
        'X-NCP-APIGW-API-KEY': naver_key         # 여기에 본인의 API 키 입력
    }

    params = {
        'start': f"{start_x},{start_y}",
        'goal': f"{end_x},{end_y}"
    }

    response = requests.get(base_url, headers=headers, params=params)
    json_data = response.json()

    if 'routes' in json_data and json_data['routes']:
        distance = json_data['routes'][0]['summary']['distance']
        print(f"출발지에서 도착지까지의 거리는 {distance}m 입니다.")
        print("경로 정보:")
        for path in json_data['routes'][0]['trafast']:
            print(path)
    else:
        print("경로를 찾을 수 없습니다.")

# 출발지와 도착지 설정 (주소)
start_address = '서울특별시 강남구 삼성동'  # 예시: '서울특별시 강남구 삼성동'
end_address = '경기도 고양시 덕양구 마상로 164'    # 예시: '서울특별시 중구 을지로'

get_directions(start_address, end_address)


서울특별시 강남구 삼성동의 위도는 37.514442, 경도는 127.062532 입니다.
경기도 고양시 덕양구 마상로 164의 위도는 37.6578214, 경도는 126.8403659 입니다.
경로를 찾을 수 없습니다.
